In [ ]:
# PASTE THIS TO THE FIRST CELL OF THE NOTEBOOK IN ORDER TO HAVE WORKING IMPORTS
import sys
import os
current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../..')) # tweak so that you get dir of code project

sys.path.append(parent_parent_dir)
print(f"Added {parent_parent_dir} to sys.path")

In [ ]:
from sklearn.cluster import OPTICS
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

## data import

In [ ]:
from src.features.get_x_y_tuples import get_x_y_tuple_list
from src.features.get_first_and_last_x_y_coordinates import get_first_and_last_x_y_coordinates 
from src.features.get_first_x_y_coordinates import get_first_x_y_coordinates
from src.features.get_last_x_y_coordinates import get_last_x_y_coordinates
def import_data(df_path):
    df_cuid = pd.read_csv(df_path)
    df_cuid_grouped_path = df_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    # 1.1 CONVERT FEATURES TO NUMBERS
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
    list_x_y_tuples = get_x_y_tuple_list(df_cuid_grouped, ['x','y'])
    first_last_x_coords, first_last_y_coords = get_first_and_last_x_y_coordinates(list_x_y_tuples)
    X = np.array([first_last_x_coords, first_last_y_coords]).T

    first_x_coords, first_y_coords = get_first_x_y_coordinates(list_x_y_tuples)
    first_coordinates = np.array([first_x_coords, first_y_coords]).T

    last_x_coords, last_y_coords = get_last_x_y_coordinates(list_x_y_tuples)
    last_coordinates = np.array([last_x_coords, last_y_coords]).T
    return df_cuid, df_cuid_grouped, X, first_coordinates, last_coordinates, list_x_y_tuples
        

In [ ]:
df_path_k729_2022 = f'{parent_parent_dir}/data/processed/k729_2022_cuid.csv'
df_path_k733_2020 = f'{parent_parent_dir}/data/processed/k733_2020_cuid.csv'
df_path_k733_2018 = f'{parent_parent_dir}/data/processed/k733_2018_cuid.csv'

df_cuid_k729_2022, df_cuid_grouped_k729_2022, X_k729_2022, first_coordinates_k729_2022, last_coordinates_k729_2022, list_x_y_tuples_k729_2022 = import_data(df_path_k729_2022)
df_cuid_k733_2020, df_cuid_grouped_k733_2020, X_k733_2020, first_coordinates_k733_2020, last_coordinates_k733_2020, list_x_y_tuples_k733_2020 = import_data(df_path_k733_2020)
df_cuid_k733_2018, df_cuid_grouped_k733_2018, X_k733_2018, first_coordinates_k733_2018, last_coordinates_k733_2018, list_x_y_tuples_k733_2018 = import_data(df_path_k733_2018)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
from src.features.calculate_path_length import calculate_path_length
from matplotlib import cm
from matplotlib.colors import Normalize

# Enable qt backend for interactive plotting
%matplotlib qt

def plot_lines_and_colored_dots(df_cuid_grouped, path_lengths, title_suffix=''):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Normalize the path lengths for coloring
    norm = Normalize(vmin=min(path_lengths), vmax=max(path_lengths))
    cmap = cm.get_cmap('viridis')

    # Loop through each path in the DataFrame
    for index, row in df_cuid_grouped.iterrows():
        x = row['x']
        y = row['y']
        z_value = path_lengths[index]  # The path length for this path
        
        # Plot the path as a line in the x-y plane (z=0 for line)
        ax.plot(x, y, zs=0, zdir='z', color='gray', alpha=0.6)
        
        # Plot colored dots along the x-y line at their respective height (z value)
        # Use the normalized path lengths for coloring
        ax.scatter(x, y, zs=[z_value] * len(x), c=[z_value] * len(x), cmap='viridis', norm=norm, marker='o', s=50)

    # Set labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Path Length (Z)')
    ax.set_title(f'3D Plot: X-Y Lines with Z-Colored Dots {title_suffix}')

    # Add a color bar to map color to path length
    mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
    mappable.set_array(path_lengths)
    fig.colorbar(mappable, ax=ax, label='Path Length (Z)')

    plt.show()

# Example usage:



In [ ]:
k729_2022_path_lengths = calculate_path_length(list_x_y_tuples_k729_2022)
plot_lines_and_colored_dots(df_cuid_grouped_k729_2022, k729_2022_path_lengths, 'k729_2022')
k733_2020_path_lengths = calculate_path_length(list_x_y_tuples_k733_2020)
plot_lines_and_colored_dots(df_cuid_grouped_k733_2020, k733_2020_path_lengths, 'k733_2020')
k733_2018_path_lengths = calculate_path_length(list_x_y_tuples_k733_2018)
plot_lines_and_colored_dots(df_cuid_grouped_k733_2018, k733_2018_path_lengths, 'k733_2018')